In [19]:
import pickle

# Define the path to the directory
directory_path = "../Datasets/sgcc_processed"

with open(directory_path+'/data_triple_mask_normalized.pkl', 'rb') as data_file:
    data = pickle.load(data_file)

with open(directory_path+'/labels.pkl', 'rb') as data_file:
    labels = pickle.load(data_file)

with open(directory_path+'/splits.pkl', 'rb') as data_file:
    splits = pickle.load(data_file)

# Print the names of the loaded dataframes

In [18]:
data.shape

(40185, 3, 1029)

# Transform data to be send in a request

The row should be converted into a dictionary where each key represent a timestamp and its value the consumption made. The algorithm is trained with 1035 days so every record to be predicted must to have this size. The first day in the data should be monday.

In [7]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'Utils')))

from data import get_dataset

file_path = '../Datasets/data.csv'

# get_dataset return the raw data ordered by time. Last column is the target variable
df_raw = get_dataset(file_path)

In [ ]:
import json

# Select the row
selected_row = df_raw.iloc[1, :]

# Convert the row to a dictionary
row_dict = selected_row.to_dict()

# Convert the dictionary to a JSON string with string keys
row_json = json.dumps({str(key): value for key, value in row_dict.items()})

In [10]:
row_json

'{"2014-01-01 00:00:00": NaN, "2014-01-02 00:00:00": NaN, "2014-01-03 00:00:00": NaN, "2014-01-04 00:00:00": NaN, "2014-01-05 00:00:00": NaN, "2014-01-06 00:00:00": NaN, "2014-01-07 00:00:00": NaN, "2014-01-08 00:00:00": NaN, "2014-01-09 00:00:00": NaN, "2014-01-10 00:00:00": NaN, "2014-01-11 00:00:00": NaN, "2014-01-12 00:00:00": NaN, "2014-01-13 00:00:00": NaN, "2014-01-14 00:00:00": NaN, "2014-01-15 00:00:00": NaN, "2014-01-16 00:00:00": NaN, "2014-01-17 00:00:00": NaN, "2014-01-18 00:00:00": NaN, "2014-01-19 00:00:00": NaN, "2014-01-20 00:00:00": NaN, "2014-01-21 00:00:00": NaN, "2014-01-22 00:00:00": NaN, "2014-01-23 00:00:00": NaN, "2014-01-24 00:00:00": NaN, "2014-01-25 00:00:00": NaN, "2014-01-26 00:00:00": NaN, "2014-01-27 00:00:00": NaN, "2014-01-28 00:00:00": NaN, "2014-01-29 00:00:00": NaN, "2014-01-30 00:00:00": NaN, "2014-01-31 00:00:00": NaN, "2014-02-01 00:00:00": NaN, "2014-02-02 00:00:00": NaN, "2014-02-03 00:00:00": NaN, "2014-02-04 00:00:00": NaN, "2014-02-05 00:00:

# Tranform data recevied in the request

Let's imagine row_json is the data received in a request

In [145]:
import pandas as pd

row_series = pd.Series(json.loads(row_json))
# Remove the Tag column
row_series = row_series[:-1]
row_series.index = pd.to_datetime(row_series.index)

# Start on monday
row_series = row_series[5:]
row_series.shape

(1029,)

In [146]:
row_series = row_series.to_frame().T
row_series.shape

(1, 1029)

In [147]:
row_series

,2014-01-06,2014-01-07,2014-01-08,2014-01-09,2014-01-10,2014-01-11,2014-01-12,2014-01-13,2014-01-14,2014-01-15,...,2016-10-22,2016-10-23,2016-10-24,2016-10-25,2016-10-26,2016-10-27,2016-10-28,2016-10-29,2016-10-30,2016-10-31
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
daysOfWeek = row_series.columns.dayofweek
daysOfWeek

Index([0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       ...
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0],
      dtype='int32', length=1029)

In [ ]:
daysOfWeek = daysOfWeek.values.reshape(1, -1)

In [150]:
daysOfWeek.shape

(1, 1029)

In [151]:
# count the number of nulls in the record
nan_count = row_series.isna().sum(axis=1)
nan_count

0    725
dtype: int64

In [152]:
nan_count[0]

725

In [153]:
#Check if the number of nulls is above the threshold
threshold = 0.71

if nan_count[0]/len(row_series.columns) > threshold:
    print('The record has too many nulls')
else:
    print('The record has an acceptable number of nulls')

The record has an acceptable number of nulls


In [154]:
import numpy as np
# 3δ principle
def process_outliers(row):
    mean = row.mean()
    std = row.std()
    threshold = mean + (3 * std)
    return pd.Series(np.where(row > threshold, threshold, row))

In [155]:
row_series.iloc[0]

2014-01-06    NaN
2014-01-07    NaN
2014-01-08    NaN
2014-01-09    NaN
2014-01-10    NaN
             ... 
2016-10-27    0.0
2016-10-28    0.0
2016-10-29    0.0
2016-10-30    0.0
2016-10-31    0.0
Name: 0, Length: 1029, dtype: float64

In [156]:
row_processed = process_outliers(row_series.iloc[0])

In [157]:
def optimized_interpolation(row):
    
    values = row.values
    result = values.copy()

    # Create a mask for NaN values
    nan_mask = np.isnan(values)

    if nan_mask[0]:
        result[0] = 0
        
    if nan_mask[-1]:
        result[-1] = 0

    # Find valid interpolation points (excluding first and last columns)
    if len(values) > 2:  # Ensure there are at least three columns to interpolate
        valid_interp = nan_mask & np.roll(~nan_mask, 1) & np.roll(~nan_mask, -1)
        valid_interp[0] = False  # Exclude first column
        valid_interp[-1] = False  # Exclude last column
        
        # Perform interpolation where valid
        result[valid_interp] = (np.roll(values, 1)[valid_interp] + np.roll(values, -1)[valid_interp]) / 2
    
    # Set remaining NaNs to 0
    result[nan_mask & ~valid_interp] = 0
    
    return pd.Series(result, index=row.index)

In [ ]:
row_processed = optimized_interpolation(row_processed)

In [159]:
def minmax_normalization(row):
    min_val = row.min()
    max_val = row.max()
    
    # Calculate the range
    range_val = max_val - min_val
    
    # Check for division by zero and apply normalization
    if range_val != 0:
        return (row - min_val) / range_val
    else:
        return pd.Series(0, index=row.index)

In [ ]:
row_processed = minmax_normalization(row_processed)

In [161]:
row_processed = row_processed.values.reshape(1, -1)

In [162]:
row_processed.shape, daysOfWeek.shape

((1, 1029), (1, 1029))

In [163]:
combined_bin = np.stack((row_processed, daysOfWeek), axis=1)
combined_bin.shape

(1, 2, 1029)

In [164]:
# Get the binary mask for the raw dataset
null_mask = row_series.isnull().astype(int)
null_mask

,2014-01-06,2014-01-07,2014-01-08,2014-01-09,2014-01-10,2014-01-11,2014-01-12,2014-01-13,2014-01-14,2014-01-15,...,2016-10-22,2016-10-23,2016-10-24,2016-10-25,2016-10-26,2016-10-27,2016-10-28,2016-10-29,2016-10-30,2016-10-31
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [165]:
null_mask.to_numpy()
null_mask.shape

(1, 1029)

In [166]:
row_processed.shape, daysOfWeek.shape, null_mask.shape

((1, 1029), (1, 1029), (1, 1029))

In [167]:
triple_mask = np.stack((row_processed, null_mask, daysOfWeek), axis = 1)
triple_mask.shape

(1, 3, 1029)

# Process single record to be predicted by the Model API

In [168]:
def process_input(record, trainingData = False):
    row_series = pd.Series(json.loads(record))

    if(trainingData):

        # Remove the Tag column (Optional - It is required for the data used in the training)
        row_series = row_series[:-1]
        row_series.index = pd.to_datetime(row_series.index)

        # Start on monday (Optional - It is required for the data used in the training)
        row_series = row_series[5:]
        row_series.shape
    
    row_series = row_series.to_frame().T

    # Get the mask of the days of the week
    daysOfWeek = row_series.columns.dayofweek
    daysOfWeek = daysOfWeek.values.reshape(1, -1)
    
    nan_count = row_series.isna().sum(axis=1)
    nan_count

    #Check if the number of nulls is above the threshold
    threshold = 0.71

    if nan_count[0]/len(row_series.columns) > threshold:
        return {
            'status': 'error',
            'message': 'The record has too many nulls. It exceeds the threshold of '+str(threshold)
        }

    row_processed = process_outliers(row_series.iloc[0])
    row_processed = optimized_interpolation(row_processed)

    row_processed = minmax_normalization(row_processed)
    row_processed = row_processed.values.reshape(1, -1)

    # Get the binary mask for the raw dataset
    null_mask = row_series.isnull().astype(int)
    null_mask

    triple_mask = np.stack((row_processed, null_mask, daysOfWeek), axis = 1)
    triple_mask.shape

    return {
        'status': 'success',
        'data': triple_mask
    }

In [173]:
record_processed = process_input(row_json, True)
record_processed['data'].shape

(1, 3, 1029)